In [1]:
import pandas as pd

In [2]:
import pandas as pd
from pathlib import Path

resampled = Path('../resampled')
coins = {}
for fp in resampled.glob('*.csv'):
    symbol = fp.stem.upper()    # e.g. "BTC"
    df = pd.read_csv(fp, parse_dates=['Date'], index_col='Date').sort_index()
    coins[symbol] = df


In [5]:
coins["BNB"]

,Price,Volume,Market_cap,Return,Log_Return,MA_7,Vol_7,MA_14,Vol_14,MA_30,Vol_30,MA_90,Vol_90
Date,,,,,,,,,,,,,
2017-09-16,0.107251,1.051223e+00,1.072506e+07,NaN,NaN,0.107251,NaN,0.107251,NaN,0.107251,NaN,0.107251,NaN
2017-09-17,0.154041,1.467859e+01,1.540413e+07,0.436274,0.362052,0.130646,NaN,0.130646,NaN,0.130646,NaN,0.130646,NaN
2017-09-18,0.173491,6.001767e+00,1.734912e+07,0.126265,0.118906,0.144928,0.219210,0.144928,0.219210,0.144928,0.219210,0.144928,0.219210
2017-09-19,0.168334,3.878927e+00,1.683342e+07,-0.029725,-0.030176,0.150779,0.237204,0.150779,0.237204,0.150779,0.237204,0.150779,0.237204
2017-09-20,0.166628,4.068762e+01,1.666279e+07,-0.010136,-0.010188,0.153949,0.215226,0.153949,0.215226,0.153949,0.215226,0.153949,0.215226
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-17,666.924447,5.698604e+08,1.026104e+11,-0.010740,-0.010798,667.802505,0.042435,629.814857,0.039263,651.576114,0.031557,677.736785,0.030850
2025-02-18,647.553994,6.508925e+08,9.963016e+10,-0.029044,-0.029475,668.336235,0.042541,635.064431,0.034677,650.384582,0.031304,678.180946,0.030979
2025-02-19,652.756991,5.785377e+08,1.004307e+11,0.008035,0.008003,661.674721,0.021775,640.993356,0.034348,649.366373,0.031350,678.522402,0.030894


In [17]:
import numpy as np

metrics = []
for sym, df in coins.items():
    if 'Market_cap' in df.columns:
        # 1. Returns
        price = df['Price']
        total_ret = price.iloc[-1]/price.iloc[0] - 1
        annual_ret = (price.iloc[-1]/price.iloc[0])**(252/len(price)) - 1

        # 2. Volatility
        logr = np.log(price).diff().dropna()
        ann_vol = logr.std() * np.sqrt(252)

        # 3. Max drawdown
        cummax = price.cummax()
        drawdown = (price - cummax) / cummax
        max_dd = drawdown.min()

        # 4. Market-cap return
        cap = df['Market_cap']
        cap_ret = cap.iloc[-1]/cap.iloc[0] - 1

        # 5. High-vol days
        vol7 = df['Vol_7']
        mv90 = vol7.rolling(90).mean()
        sv90 = vol7.rolling(90).std()
        high_vol_pct = ((vol7 > mv90 + 2*sv90).sum() / len(vol7)) * 100

        metrics.append({
            'Coin': sym,
            'TotalReturn': total_ret,
            'AnnReturn': annual_ret,
            'AnnVol': ann_vol,
            'MaxDrawdown': max_dd,
            'CapReturn': cap_ret,
            'HighVolDays%': high_vol_pct
        })

summary = pd.DataFrame(metrics).set_index('Coin').round(4)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(summary)


         TotalReturn  AnnReturn  AnnVol  MaxDrawdown    CapReturn  \
Coin                                                                
BTC         127.1420     0.3499  0.5801      -0.8364     212.0284   
SOLANA      175.4144     1.0816  1.0581      -0.9626   10544.1574   
LIDO          3.4468     0.2805  0.6534      -0.8052    5371.7741   
USDC         -0.0371    -0.0041  0.0462      -0.0693    2312.0839   
RIPPLE      663.1265     0.5189  0.9614      -0.9594    4380.8211   
ETH        1996.6409     0.7322  0.8727      -0.9421    3988.3546   
CARDANO      27.3289     0.3688  0.9519      -0.9799      40.0867   
TRON         98.5671     0.5458  0.9889      -0.9632     129.7998   
BNB        6031.6585     1.2427  1.5028      -0.8339    9280.6157   
DOGE        659.9281     0.5189  1.0674      -0.9221    1211.5320   
THETER       -0.1739    -0.0131  0.2771      -0.5269  466964.0629   

         HighVolDays%  
Coin                   
BTC            7.0903  
SOLANA         5.9055  
LIDO  